# Balance reaction mass and charge

### Find reactions that have unbalanced charges, and add protons etc. as necessary to repair.


example scripts:
https://github.com/ChristianLieven/memote-m-capsulatus/blob/master/ComplementaryScripts/12.%20ApplyMemoteDrivenCuration.ipynb

https://github.com/SysBioChalmers/Sco-GEM/blob/devel/code/curation/v1_3_2.py


especially relevant for carveme models:
https://github.com/cdanielmachado/carveme/issues/136

use alongside custom reaction databases?
https://github.com/SysBioChalmers/Salb-GEM/tree/master/ComplementaryData/curation

Import libraries, read in SBML v3 model

In [4]:
import cobra
import os
import numpy as np
import pandas as pd
import memote.support.consistency as consistency
import memote.support.consistency_helpers as con_helpers
from memote.utils import annotate, get_ids, truncate, wrapper
from six import iteritems, itervalues
from collections import defaultdict
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
model_in = cobra.io.read_sbml_model('/projectnb2/talbot-lab-data/metabolic_models/genomes/solibacter/solibacter.xml')

Identify problematic reactions and their metaNET listings from annotations (https://github.com/opencobra/cobrapy/issues/684)

In [9]:
rxn_list = consistency.find_mass_unbalanced_reactions(model_in.reactions)
rxn_list = consistency.find_charge_unbalanced_reactions(model_in.reactions) # handle mass & charge simultaneously?
rxn = rxn_list[0]
print(rxn)
rxn.annotation.get['metanetx'] # ???
#rxn.check_mass_balance()
#charge = model_in.reactions.rxn.check_mass_balance()
#print(charge)
#model.reactions.PAMMOipp.add_metabolites({model.metabolites.h_p:1}, combine= True)

2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> amp_c + pg160_c + ppi_c


TypeError: 'builtin_function_or_method' object is not subscriptable

Fix mass and charge balances for one reaction (testing)

In [7]:
rxn = rxn_list[0]
print(rxn)

rxn.check_mass_balance()
#charge = model_in.reactions.rxn.check_mass_balance()
#print(charge)
#model.reactions.PAMMOipp.add_metabolites({model.metabolites.h_p:1}, combine= True)

charge = 0
for metabolite, coefficient in iteritems(rxn.metabolites):
    charge += coefficient * metabolite.charge
print(charge)
print(con_helpers.is_charge_balanced(rxn))


2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> amp_c + pg160_c + ppi_c


{'charge': 1.0}

1.0
False


### Run loop on all reactions

Apply to all problematic reactions and evaluate success

In [8]:
for rxn in rxn_list:
    print(rxn)
    #charge = model_in.reactions.rxn.check_mass_balance()
    #print(charge)

    charge = 0
    for metabolite, coefficient in iteritems(rxn.metabolites):
        charge += coefficient * metabolite.charge
    print(charge)
    print(con_helpers.is_charge_balanced(rxn))
    #model.reactions.PAMMOipp.add_metabolites({model.metabolites.h_p:1}, combine= True)

    

2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> amp_c + pg160_c + ppi_c
1.0
False
2AGPGAT161: 2agpg161_c + atp_c + hdcea_c --> amp_c + pg161_c + ppi_c
-1.0
False
2DDARAA: 2ddara_c <=> gcald_c + pyr_c
-1.0
False
2MBALDH: 2mbald_c + h2o_c + nad_c <=> 2mba_c + 2.0 h_c + nadh_c
1.0
False
2S6HCCi: akg_c + h_c + ichor_c <=> 2shchc_c + co2_c + pyr_c
2.0
False
3HAACOAT80: 3hoctACP_c + coa_c <=> ACP_c + R_3hocoa_c
6.0
False
A6PAG: atp_c + gal_c --> adp_c + dgal6p_c + h_c
2.0
False
AACPS2: ACP_c + atp_c + ttdcea_c --> amp_c + ppi_c + tdeACP_c
-2.0
False
AACPS4: ACP_c + atp_c + hdcea_c --> amp_c + hdeACP_c + ppi_c
-2.0
False
AACPS7: ACP_c + atp_c + ddca_c --> amp_c + ddcaACP_c + ppi_c
-1.0
False
AACPS9: ACP_c + atp_c + octa_c --> amp_c + ocACP_c + ppi_c
-2.0
False
AALDH: nadh_c + pacald_c <=> nad_c + pea_c
1.0
False
ACACT5r_1: accoa_c + dccoa_c <=> 3oddcoa_c + coa_c
-4.0
False
ACACT6r_1: accoa_c + ddcoa_c <=> 3otdcoa_c + coa_c
-4.0
False
ACGAMPM: acgam6p_c <=> acgam1p_c
2.0
False
ACHBS: 2obut_c + h_c

In [ ]:
for rxn in rxn_list:
    print(rxn)
    balance = defaultdict(int)
    for metabolite, coefficient in iteritems(rxn.metabolites):
        for element, amount in iteritems(metabolite.elements):
            balance[element] += coefficient * amount
    #print(balance.values)
    h_val = balance.pop('H')
    print(h_val)
    balance = rxn.check_mass_balance()
    print(balance)
    rxn.add_metabolites({model.model_in.h_p:-1}, combine= True)
    rxn.check_mass_balance()

    #if h_val == 1:
    #    rxn.add_metabolites({model.metabolites.h_p:1}, combine= True)
    
#model_in.reactions.SALCHS4FEabcpp.check_mass_balance()
#model.reactions.NO3t2pp.add_metabolites({model.metabolites.h_p:1}, combine= True)
    
    